In [ ]:
import numpy as np
from IPython.html.widgets import *
import matplotlib.pyplot as plt
import datetime
from mpl_toolkits.basemap import Basemap, shiftgrid
import pygrib
import glob
import string
from shapely.geometry import Point, mapping
from fiona import collection
from IPython.html import widgets
from nefis2 import *

from grid import *
from dep import *


#%matplotlib inline

In [ ]:
from gribapi import *
def readgrib(INPUT):
    f = open(INPUT)

#    for t in range(3*time):
#        gid = grib_new_from_file(f,headers_only = True)
    
    for l in range(1):
        gid = grib_new_from_file(f)
        if gid is None: break

        name=grib_get(gid, 'shortName')
        
        ni = grib_get(gid, 'Ni')

        nj =  grib_get(gid, 'Nj')

        lat=grib_get_array(gid,'latitudes')
        lon=grib_get_array(gid,'longitudes')

        dat=grib_get_array(gid,'values')

        grib_release(gid)

    f.close()

    return dat.reshape(nj,ni),lat.reshape(nj,ni),lon.reshape(nj,ni)


In [ ]:
def mplot(lats,lons,var,title): 
        
    # define projection window

    llcrnrlat=lats.min()
    llcrnrlon=lons.min()
    urcrnrlat=lats.max()
    urcrnrlon=lons.max()

    
    # make orthographic basemap.
    m = Basemap(projection='cyl',llcrnrlat=llcrnrlat,urcrnrlat=urcrnrlat,\
             llcrnrlon=llcrnrlon,urcrnrlon=urcrnrlon,resolution='h')
    # create figure, add axes
    fig1 = plt.figure(figsize=(8,10))
    ax = fig1.add_axes([0.1,0.1,0.8,0.8])
    # set desired contour levels.
    clevs = np.linspace(var.min(),var.max(), 10,endpoint=True)
    
    
    # compute native x,y coordinates of grid.
    x, y = m(lons, lats)
    # define parallels and meridians to draw.
    parallels = np.arange(-90.,90,10.)
    meridians = np.arange(0.,360.,10.)
    # plot SLP contours.
    CS1 = m.contour(x,y,var,clevs,linewidths=0.5,colors='k',animated=True)
    CS2 = m.contourf(x,y,var,clevs,cmap=plt.cm.RdBu_r,animated=True)
    # draw coastlines, parallels, meridians.
    m.drawcoastlines(linewidth=1.5)
    m.drawparallels(parallels)
    m.drawmeridians(meridians)
    # add colorbar
    cb = m.colorbar(CS2,"right", size="5%", pad="2%", ticks=clevs)
    cb.set_label('m')
    # set plot title
    ax.set_title(title)
    
    plt.show()

In [ ]:
def wmap(yyyy=2015,mm=6,dd=3,hh=0,time=1):
    hh=np.int(hh) # The variables are passed as string and needs to become integer to be used later
    yyyy=np.int(yyyy)
    mm=np.int(mm)
    dd=np.int(dd)
    time=np.int(time)
    # specify date to plot.
    date = datetime.datetime(yyyy,mm,dd,hh)

    # set PATH of the database.
    PATHbase="/mnt/Tsunamiweb/grib_HNMS_WAV/"  # Local mapping location for the above network drive
  # PATHbase="/mnt/ECMWF/grib/"  # Local mapping location for the above network drive
    PATH=PATHbase+"%04i/%02i/%02i/" % (yyyy,mm,dd)

    dpath=glob.glob(PATH+'*_{:04d}_{:02d}_{:02d}_{:02d}00_{:02d}h.grb'.format(yyyy,mm,dd,hh,time))

    try:
        data = pygrib.open(dpath[0])
    #    data,lats,lons = readgrib(dpath[0]) 
    except:        
        print 'no available data in ', PATH
        return

    # read lats,lons
    latlons=data[1].latlons()
    # reverse latitudes so they go from south to north.
    lats = np.flipud(latlons[0]) # make them increasing
    lons = latlons[1]
 #   lats=np.flipud(lats)
    # get significant height
    sh = data[1].data()[0]
 #   sh = np.ma.masked_equal(data,9999.)
    shi =np.flipud(sh) # make them increasing in lat

    title = 'Significant height of combined wind waves and swell\n Forecast {} from analysis on {}\n'.format(time,datetime.datetime.strftime(date, '%Y%m%d_%H')) + \  
                 datetime.datetime.strftime(date+datetime.timedelta(hours=time), '%a %b %d  %H:%M:%S %Z %Y')
        
    mplot(lats,lons,shi,title)


In [ ]:
    # define projection window
    
    llcrnrlat=lats.min()
    llcrnrlon=lons.min()
    urcrnrlat=lats.max()
    urcrnrlon=lons.max()

    
    # make orthographic basemap.
    m = Basemap(projection='cyl',llcrnrlat=llcrnrlat,urcrnrlat=urcrnrlat,\
             llcrnrlon=llcrnrlon,urcrnrlon=urcrnrlon,resolution='h')
    # create figure, add axes
    fig1 = plt.figure(figsize=(8,10))
    ax = fig1.add_axes([0.1,0.1,0.8,0.8])
    # set desired contour levels.
    clevs = 20

    # compute native x,y coordinates of grid.
    x, y = m(lons, lats)
    # define parallels and meridians to draw.
    parallels = np.arange(-90.,90,10.)
    meridians = np.arange(0.,360.,10.)
    # plot SLP contours.
    CS1 = m.contour(x,y,shi,clevs,linewidths=0.5,colors='k',animated=True)
    CS2 = m.contourf(x,y,shi,clevs,cmap=plt.cm.RdBu_r,animated=True)
    # draw coastlines, parallels, meridians.
    m.drawcoastlines(linewidth=1.5)
    m.drawparallels(parallels)
    m.drawmeridians(meridians)
    # add colorbar
    cb = m.colorbar(CS2,"right", size="5%", pad="2%")
    cb.set_label('m')
    # set plot title
    ax.set_title('Significant height of combined wind waves and swell\n Forecast {} from analysis on {}\n'.format(time,datetime.datetime.strftime(date, '%Y%m%d_%H')) + \  
                 datetime.datetime.strftime(date+datetime.timedelta(hours=time), '%a %b %d  %H:%M:%S %Z %Y'))
    
    plt.show()

In [ ]:
def wmap1(yyyy=2015,mm=6,dd=3,hh=0,time=1):
    hh=np.int(hh) # The variables are passed as string and needs to become integer to be used later
    yyyy=np.int(yyyy)
    mm=np.int(mm)
    dd=np.int(dd)
    time=np.int(time)
    # specify date to plot.
    date = datetime.datetime(yyyy,mm,dd,hh)

    # set PATH of the database.
    PATH="/mnt/delft3d/DELFT3D/"  # Local mapping location for the above network drive
    
    d = Nefis(PATH+'waves/wavm-med.dat',PATH+'waves/wavm-med.def',access='r')
    
    try:
        
      h = d.getdata('map-series','HSIGN')
      u = d.getdata('WIND','WINDU')
      v = d.getdata('WIND','WINDV')
      wu = d.getdata('map-series','VELOC-X')
      wv = d.getdata('map-series','VELOC-Y')
      xp = d.getdata('map-series','XP')
      yp = d.getdata('map-series','YP')
      t=d.getdata('map-series','TIME')[:]

    except:        
        print 'no available data in ', PATH
        return
      
    h = np.array(h)
    u = np.array(u)
    v = np.array(v)
    lons = np.array(xp)[0,:,:]
    lats = np.array(yp)[0,:,:]
    t = np.array(t).flatten()

    # define projection window
    
    llcrnrlat=lats.min()
    llcrnrlon=lons.min()
    urcrnrlat=lats.max()
    urcrnrlon=lons.max()

    
    # make orthographic basemap.
    m = Basemap(projection='cyl',llcrnrlat=llcrnrlat,urcrnrlat=urcrnrlat,\
             llcrnrlon=llcrnrlon,urcrnrlon=urcrnrlon,resolution='h')
    # create figure, add axes
    fig1 = plt.figure(figsize=(8,10))
    ax = fig1.add_axes([0.1,0.1,0.8,0.8])
    # set desired contour levels.
    clevs = 20

    # compute native x,y coordinates of grid.
    x, y = m(lons, lats)
    # define parallels and meridians to draw.
    parallels = np.arange(-90.,90,10.)
    meridians = np.arange(0.,360.,10.)
    
    hh=h[time/3,:,:]

    grd=Grid.fromfile(folder+'med.grd')
    deb=Dep.read(folder+'med.dep',grd.shape)
    b=deb.val[:-1,:-1]
    w=b<0.
    #w=w.T
    bh=np.ma.masked_where(w==True,hh)

    
    # plot SLP contours.
    CS1 = m.contour(x,y,bh,clevs,linewidths=0.5,colors='k',animated=True)
    CS2 = m.contourf(x,y,bh,clevs,cmap=plt.cm.RdBu_r,animated=True)
    # draw coastlines, parallels, meridians.
    m.drawcoastlines(linewidth=1.5)
    m.drawparallels(parallels)
    m.drawmeridians(meridians)
    # add colorbar
    cb = m.colorbar(CS2,"right", size="5%", pad="2%")
    cb.set_label('m')
    # set plot title
    ax.set_title('Significant height of combined wind waves and swell\n Forecast {} from analysis on {}\n'.format(time,datetime.datetime.strftime(date, '%Y%m%d_%H')) +\
                 datetime.datetime.strftime(date+datetime.timedelta(hours=time), '%a %b %d  %H:%M:%S %Z %Y'))
    
    plt.show()

In [ ]:
def compare(yyyy=2015,mm=6,dd=3,hh=0,time=1):
    wmap(yyyy,mm,dd,hh,time)
    wmap1(yyyy,mm,dd,hh,time)

In [ ]:
# Define range of file attributes
yy=[2016]
yy=[w for w in map(str,yy )]
mm=np.arange(1,13)
mm=[w for w in map(str,mm )]
dd=np.arange(1,32)
dd=[w for w in map(str,dd )]
hh=[0,12]
hh=[w for w in map(str,hh )]
tt=np.arange(0,51,3)
tt=[w for w in map(str,tt )]



In [ ]:
interact_manual(compare, yyyy=yy, mm=mm, dd=dd, hh=hh, time=tt)